In [ ]:
# | default_exp _code_generator.test_generator

In [ ]:
# | export

from typing import *
import time
import importlib.util
from tempfile import TemporaryDirectory
from pathlib import Path
import platform
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    CustomAIChat,
    ValidateAndFixResponse,
    write_file_contents,
    read_file_contents,
    validate_python_code,
)
from faststream_gen._code_generator.prompts import TEST_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    APPLICATION_FILE_NAME,
    INTEGRATION_TEST_FILE_NAME,
)

In [ ]:


from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _validate_response(test_code: str, **kwargs: str) -> List[str]:
    with TemporaryDirectory() as d:
        write_file_contents(f"{d}/{APPLICATION_FILE_NAME}", kwargs["app_code"])
        
        test_file = f"{d}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(test_file, test_code)

        cmd = ["pytest", test_file, "--tb=short"]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        if p.returncode != 0:
            return [str(p.stdout.decode('utf-8'))]

        return []

In [ ]:
fixture_test_code = """
def test_always_passes():
    assert True
"""
kwargs = {
    "app_code": "print('hi')"
}
expected = []
actual = _validate_response(fixture_test_code, **kwargs)
print(actual)
assert actual == expected

[]


In [ ]:
fixture_test_code = """
def test_always_fails():
    assert False
"""
kwargs = {
    "app_code": "print('hi')"
}
actual = _validate_response(fixture_test_code, **kwargs)
print(actual[0])
assert actual != []
print("OK")

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmpjrvnava8
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmpjrvnava8/test.py F                                       [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________
/tmp/tmpjrvnava8/test.py:3: in test_always_fails
    assert False
E   assert False
=========================== short test summary info ============================
FAILED ../../../tmp/tmpjrvnava8/test.py::test_always_fails - assert False
============================== 1 failed in 0.04s ===============================

OK


In [ ]:
# fixture_test_code = """
# import asyncio
# from fastkafka.testing import Tester
# try:
#     from .application import *
# except ImportError as e:
#     from application import *


# async def async_tests():
#     async with Tester(change_currency_app) as tester:
#         input_msg = StoreProduct(product_name="Test Product", currency="HRK", price=100.0)

#         # tester consumes message from the store_product topic
#         await tester.on_store_product(input_msg)

#         # assert that tester consumed from the store_product topic and it was called with the accurate argument
#         await tester.awaited_mocks.on_store_product.assert_called_with(
#             input_msg, timeout=5
#         )

#         # assert that tester produced message to the change_currency topic and it was called with the accurate argument
#         await tester.awaited_mocks.to_change_currency.assert_called_with(
#             StoreProduct(product_name="Test Product", currency="EUR", price=13.333333333333334), timeout=5
#         )
#     print("ok")


# if __name__ == "__main__":
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(async_tests())

# """

# kwargs = {
#     "app_code": """
# from typing import *
# from pydantic import BaseModel, Field
# from aiokafka.helpers import create_ssl_context

# from fastkafka import FastKafka


# class StoreProduct(BaseModel):
#     product_name: str = Field(..., description="Name of the product.")
#     currency: str = Field(..., description="Currency of the product.", pattern="^[A-Z]{3}$")
#     price: float = Field(..., description="Price of the product.")

# kafka_brokers = {
#     "localhost": {
#         "url": "localhost",
#         "description": "local development kafka broker",
#         "port": 9092,
#     },
#     "staging": {
#         "url": "staging.airt.ai",
#         "description": "staging kafka broker",
#         "port": 9092,
#         "protocol": "kafka-secure",
#         "security": {"type": "scramSha256"},
#     },
#     "production": {
#         "url": "prod.airt.ai",
#         "description": "production kafka broker",
#         "port": 9092,
#         "protocol": "kafka-secure",
#         "security": {"type": "scramSha256"},
#     }
# }

# change_currency_app_description = "Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number. It should consume messages from 'store_product' topic and the message will be a JSON encoded object with three attributes: product_name, currency, and price. For each consumed message, check if the currency attribute is set to 'HRK'. If it is, change the currency to 'EUR' and divide the price by 7.5. If the currency is not set to 'HRK', the original message remains unchanged. Finally, publish the consumed message to 'change_currency' topic."

# change_currency_app = FastKafka(
#     kafka_brokers=kafka_brokers, 
#     description=change_currency_app_description, 
#     version="0.0.1", 
#     title='Change Currency',
#     security_protocol = "SASL_SSL",
#     sasl_mechanism= "SCRAM-SHA-256",
#     sasl_plain_username= "<username>",
#     sasl_plain_password=  "<password>",
#     ssl_context= create_ssl_context(),
# )


# store_product_description = "For each consumed message, check if the currency attribute is set to 'HRK'. If it is, change the currency to 'EUR' and divide the price by 7.5. If the currency is not set to 'HRK', the original message remains unchanged. Finally, publish the consumed message to 'change_currency' topic."

# @change_currency_app.consumes(topic="store_product", description=store_product_description)
# async def on_store_product(msg: StoreProduct):
#     if msg.currency == "HRK":
#         msg.currency = "EUR"
#         msg.price /= 7.5
#     await to_change_currency(msg)


# change_currency_description = "Produce the incoming messages to the 'change_currency' topic."
# @change_currency_app.produces(topic="change_currency", description=change_currency_description)
# async def to_change_currency(msg: StoreProduct) -> StoreProduct:
#     return msg

# """
# }

# expected = 'AttributeError: \'Tester\' object has no attribute \'on_store_product\''
# actual = _validate_response(fixture_test_code, **kwargs)
# print(actual)
# assert expected in "".join(actual)

In [ ]:
# | export


def generate_test(
    description: str,
    code_gen_directory: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> List[Dict[str, int]]:
    """Generate integration test for the FastStream app

    Args:
        description: Validated User application description
        code_gen_directory: The directory containing the generated files.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The generated integration test code for the application
    """
    with yaspin(text="Generating tests...", color="cyan", spinner="clock") as sp:
        app_file_name = f"{code_gen_directory}/{APPLICATION_FILE_NAME}"
        app_code_prompt = read_file_contents(app_file_name)

        prompt = (
            TEST_GENERATION_PROMPT.replace(
                "==== REPLACE WITH APP DESCRIPTION ====", description
            )
            .replace("==== RELEVANT EXAMPLES GOES HERE ====", relevant_prompt_examples)
            .replace("from .app import", "from application import")
        )
        test_generator = CustomAIChat(
            user_prompt=prompt,
            semantic_search_query="How to test FastStream applications? Explain in detail.",
        )
        test_validator = ValidateAndFixResponse(test_generator, _validate_response)
        validated_test, total_usage = test_validator.fix(
            f"{prompt}\n{app_code_prompt}",
            total_usage=total_usage,
            app_code=app_code_prompt,
        )

        output_file = f"{code_gen_directory}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(output_file, validated_test)

        sp.text = ""
        sp.ok(f" ✔ Tests are generated and saved at: {output_file}")
        return total_usage

In [ ]:
# | notest

fixture_code = """
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(
        ..., examples=["Apple"], description="Product name example"
    )
    currency: str = Field(
        ..., examples=["HRK"], description="Currency example"
    )
    price: float = Field(
        ..., examples=[10.0], description="Price example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(msg: Product, logger: Logger) -> Product:
    logger.info(msg)

    if msg.currency == "HRK":
        logger.info(f"Changing currency and price for {msg.product_name}")
        msg = Product(product_name=msg.product_name, currency="EUR", price = msg.price / 7.5)

    return msg
"""

fixture_description = """
Create a FastStream application using localhost broker for testing and use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'. For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.
"""

relevant_examples = """
==== EXAMPLE APP CODE ====

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker
from typing import Optional


class CourseUpdates(BaseModel):
    course_name: str = Field(
        ..., examples=["Biology"], description="Course example"
    )
    new_content: Optional[str] = Field(
        default=None, examples=["New content"], description="Content example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("notify_update")
@broker.subscriber("course_updates")
async def on_course_update(msg: CourseUpdates, logger: Logger) -> CourseUpdates:
    logger.info(msg)

    if msg.new_content:
        logger.info(f"Course has new content {msg.new_content=}")
        msg = CourseUpdates(course_name=("Updated: " + msg.course_name), new_content=msg.new_content)
    return msg

==== YOUR RESPONSE ====

import pytest

from faststream.kafka import TestKafkaBroker

from application import *


@broker.subscriber("notify_update")
async def on_notify_update(msg: CourseUpdates):
    pass

@pytest.mark.asyncio
async def test_app():
    async with TestKafkaBroker(broker):
        await broker.publish(CourseUpdates(course_name="Biology"), "course_updates")
        on_course_update.mock.assert_called_with(dict(CourseUpdates(course_name="Biology")))
        on_notify_update.mock.assert_called_with(dict(CourseUpdates(course_name="Biology")))

        await broker.publish(CourseUpdates(course_name="Biology", new_content="We have additional classes..."), "course_updates")
        on_course_update.mock.assert_called_with(dict(CourseUpdates(course_name="Biology", new_content="We have additional classes...")))
        on_notify_update.mock.assert_called_with(dict(CourseUpdates(course_name="Updated: Biology", new_content="We have additional classes...")))

"""

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    test_file = f"{output_path}/{APPLICATION_FILE_NAME}"    
    write_file_contents(test_file, fixture_code)
    
    usage = generate_test(fixture_description, output_path, [], relevant_examples)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    output_file = f"{output_path}/{INTEGRATION_TEST_FILE_NAME}"
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

⠋ Generating tests...relevant_prompt_examples='\n==== EXAMPLE APP CODE ====\n\nfrom pydantic import BaseModel, Field\n\nfrom faststream import FastStream, Logger\nfrom faststream.kafka import KafkaBroker\nfrom typing import Optional\n\n\nclass CourseUpdates(BaseModel):\n    course_name: str = Field(\n        ..., examples=["Biology"], description="Course example"\n    )\n    new_content: Optional[str] = Field(\n        default=None, examples=["New content"], description="Content example"\n    )\n\n\nbroker = KafkaBroker("localhost:9092")\napp = FastStream(broker)\n\n\n@broker.publisher("notify_update")\n@broker.subscriber("course_updates")\nasync def on_course_update(msg: CourseUpdates, logger: Logger) -> CourseUpdates:\n    logger.info(msg)\n\n    if msg.new_content:\n        logger.info(f"Course has new content {msg.new_content=}")\n        msg = CourseUpdates(course_name=("Updated: " + msg.course_name), new_content=msg.new_content)\n    return msg\n\n==== YOUR RESPONSE ====\n\nimpor

⠹ Generating tests...

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:120: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


⠏ Generating tests... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmphnzp9519
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmphnzp9519/test.py F                                       [100%]

=================================== FAILURES ===================================
___________________________________ test_app ___________________________________
/tmp/tmphnzp9519/test.py:21: in test_app
    on_change_currency.mock.assert_not_called()
/usr/lib/python3.11/unittest/mock.py:900: in assert_not_called
    raise AssertionError(msg)
E   AssertionError: Expected 'mock' to not have been called. Called 2 times.
E   Calls: [call({'product_name': 'Apple', 'currency': 'EUR', 'price': 1.3333333333333333}),
E    call({'product_name':

⠸ Generating tests... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmp34prkzjb
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmp34prkzjb/test.py F                                       [100%]

=================================== FAILURES ===================================
___________________________________ test_app ___________________________________
/tmp/tmp34prkzjb/test.py:21: in test_app
    on_change_currency.mock.assert_not_called()
/usr/lib/python3.11/unittest/mock.py:900: in assert_not_called
    raise AssertionError(msg)
E   AssertionError: Expected 'mock' to not have been called. Called 2 times.
E   Calls: [call({'product_name': 'Apple', 'currency': 'EUR', 'price': 1.3333333333333333}),
E    call({'product_name':

⠧ Generating tests... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmp_qexbbee
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmp_qexbbee/test.py F                                       [100%]

=================================== FAILURES ===================================
___________________________________ test_app ___________________________________
/tmp/tmp_qexbbee/test.py:21: in test_app
    on_change_currency.mock.assert_not_called()
/usr/lib/python3.11/unittest/mock.py:900: in assert_not_called
    raise AssertionError(msg)
E   AssertionError: Expected 'mock' to not have been called. Called 2 times.
E   Calls: [call({'product_name': 'Apple', 'currency': 'EUR', 'price': 1.3333333333333333}),
E    call({'product_name':

⠸ Generating tests... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmpni23rh69
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmpni23rh69/test.py F                                       [100%]

=================================== FAILURES ===================================
___________________________________ test_app ___________________________________
/tmp/tmpni23rh69/test.py:21: in test_app
    on_change_currency.mock.assert_not_called()
/usr/lib/python3.11/unittest/mock.py:900: in assert_not_called
    raise AssertionError(msg)
E   AssertionError: Expected 'mock' to not have been called. Called 2 times.
E   Calls: [call({'product_name': 'Apple', 'currency': 'EUR', 'price': 1.3333333333333333}),
E    call({'product_name':

⠴ Generating tests... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmp8x7qen_3
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmp8x7qen_3/test.py F                                       [100%]

=================================== FAILURES ===================================
___________________________________ test_app ___________________________________
/tmp/tmp8x7qen_3/test.py:13: in test_app
    on_change_currency.mock.assert_called_with(dict(Product(product_name="Apple", currency="EUR", price=1.3333333333333333)))
E   NameError: name 'on_change_currency' is not defined
----------------------------- Captured stdout call -----------------------------
2023-09-08 10:35:09,452 INFO     - Received
2023-09-08 10:35:09,453 INFO  

ValueError: ✘ Error: Maximum number of retries (5) exceeded. Unable to fix the following issues. Please try again...
[1m============================= test session starts ==============================[0m
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmp8x7qen_3
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmp8x7qen_3/test.py [31mF[0m[31m                                       [100%][0m

=================================== FAILURES ===================================
[31m[1m___________________________________ test_app ___________________________________[0m
[1m[31m/tmp/tmp8x7qen_3/test.py[0m:13: in test_app
    on_change_currency.mock.assert_called_with([96mdict[39;49;00m(Product(product_name=[33m"[39;49;00m[33mApple[39;49;00m[33m"[39;49;00m, currency=[33m"[39;49;00m[33mEUR[39;49;00m[33m"[39;49;00m, price=[94m1.3333333333333333[39;49;00m)))[90m[39;49;00m
[1m[31mE   NameError: name 